In [1]:
import warnings
warnings.filterwarnings('ignore')

import geopandas
import r5py
import shapely
import pandas as pd
import geopandas as gpd
import numpy as np
import datetime

import os
import snman
from snman import osmnx_customized as oxc
from snman.constants import *

PERIMETER = '_accessibility_debug'

# Set these paths according to your own setup
data_directory = os.path.join('C:',os.sep,'Users','lballo','polybox','Research','SNMan','SNMan Shared','data_v2')
inputs_path = os.path.join(data_directory, 'inputs')
process_path = os.path.join(data_directory, 'process', PERIMETER)
outputs_path = os.path.join(data_directory, 'outputs', PERIMETER)

CRS_internal = 2056      # for Zurich

In [2]:
print ('import statent grid')
statent = gpd.read_parquet(
    os.path.join(inputs_path, 'switzerland', 'switzerland', 'statent', 'STATENT_2021_ebc_10_reduced_fields.gzip')
)
statent.rename(columns={'RELI': 'id'}, inplace=True)

print('read statpop that has been pre-joined with statent')
statpop = gpd.read_parquet(
    os.path.join(inputs_path, 'switzerland', 'switzerland', 'statpop', 'statpop2017_with_statent_reduced_columns.gzip')
)

import statent grid
read statpop that has been pre-joined with statent


In [3]:
print('read transport network default')
transport_network_default = r5py.TransportNetwork(
    os.path.join(outputs_path, 'before_oneway_links_default.osm.pbf'),
    #os.path.join(inputs_path, 'switzerland', 'switzerland', 'gtfs', 'gtfs_fp2024_2024-06-27_mod.zip')
    #os.path.join(inputs_path, 'switzerland', 'switzerland', 'gtfs', 'vbz_2024.zip')
)

read transport network default


In [4]:
print('read transport network cycling')
transport_network_cycling = r5py.TransportNetwork(
    os.path.join(outputs_path, 'before_oneway_links_cycling.osm.pbf'),
)

read transport network cycling


In [5]:
print('read transport network cycling after')
transport_network_cycling_after = r5py.TransportNetwork(
    os.path.join(outputs_path, 'after_oneway_links_cycling.osm.pbf'),
)

read transport network cycling after


In [6]:
print('read transport network car')
transport_network_car = r5py.TransportNetwork(
    os.path.join(outputs_path, 'tt.osm.pbf'),
)

read transport network car


In [7]:
perimeter = snman.io.import_geofile_to_gdf(
    os.path.join(inputs_path, 'perimeters', 'perimeters.shp'), index='id'
).geometry['ebc_zrh_v01'].simplify(500, preserve_topology=True)
statent_in_perimeter = statent[statent.within(perimeter)]

In [8]:
cells = list(statent_in_perimeter['id'].sample(n=10, random_state=0))

def miniwrapper(cell):
    print(f'calculating cell {cell}')
    return snman.accessibility.calculate_accessibility_for_statent_cell(
        transport_network_default,
        transport_network_car,
        transport_network_cycling,
        statpop, statent, cell,
        distance_limit=50*1000,
        population_sample=0.05,
        destinations_sample=0.02
    )

accessibility_before = pd.concat(
    map(miniwrapper, cells),
    ignore_index=True
)

print('exporting file')
snman.io.export_gdf(
    accessibility_before,
    os.path.join(outputs_path, 'accessibility_before.gpkg')
)

calculating cell 70042281
calculating cell 68132486
calculating cell 68672447
calculating cell 69702513
calculating cell 67962611
calculating cell 66382604
calculating cell 68322416
calculating cell 66682596
calculating cell 68172475
calculating cell 69482396
exporting file


In [9]:
cells = list(statent_in_perimeter['id'].sample(n=10, random_state=0))

def miniwrapper(cell):
    print(f'calculating cell {cell}')
    return snman.accessibility.calculate_accessibility_for_statent_cell(
        transport_network_default,
        transport_network_car,
        transport_network_cycling_after,
        statpop, statent, cell,
        distance_limit=50*1000,
        population_sample=0.05,
        destinations_sample=0.02
    )

accessibility_after = pd.concat(
    map(miniwrapper, cells),
    ignore_index=True
)

print('exporting file')
snman.io.export_gdf(
    accessibility_after,
    os.path.join(outputs_path, 'accessibility_after.gpkg')
)

calculating cell 70042281
calculating cell 68132486
calculating cell 68672447
calculating cell 69702513
calculating cell 67962611
calculating cell 66382604
calculating cell 68322416
calculating cell 66682596
calculating cell 68172475
calculating cell 69482396
exporting file


In [10]:
accessibility_keys = [
    'accessibility', 'accessibility_private_cars',
    'accessibility_cycling', 'accessibility_foot', 'accessibility_transit'
]

accessibility_before_after = pd.merge(
    accessibility_before,
    accessibility_after[accessibility_keys],
    left_index=True, right_index=True,
    suffixes=['_before', '_after']
)

for key in accessibility_keys:

    accessibility_before_after[f'{key}_after-before'] = (
            accessibility_before_after[f'{key}_after']
            - accessibility_before_after[f'{key}_before']
    )

snman.io.export_gdf(
    accessibility_before_after,
    os.path.join(outputs_path, 'accessibility_before_after.gpkg')
)

In [11]:
if 0:
    import warnings
    warnings.filterwarnings('ignore')
    
    accessibility_contributions = snman.accessibility.calculate_accessibility_for_statent_cell(
        transport_network_default,
        transport_network_car,
        transport_network_cycling_after,
        statpop, statent, 69602294,
        distance_limit=50*1000,
        population_sample=0.05,
        destinations_sample=0.0166,
        sum_accessibility_contributions=False
    )
    
    snman.io.export_gdf(
        accessibility_contributions,
        os.path.join(outputs_path, 'accessibility_contributions.gpkg')
    )